In [1]:
# # 패키지 설치
# !pip install ijson
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver --yes
# !pip install pororo

In [2]:
import ijson
import pandas as pd
import re
import numpy as np
import time
from tqdm import tqdm

import pororo
from pororo import Pororo

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# 데이터 불러오기
train = pd.read_csv('./data/new_train_data.csv')
test = pd.read_csv('./data/test_data.csv')

In [4]:
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
27993,2995,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사는 중단없이 마무리 되었다.,contradiction
27994,2996,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사가 중단된 건 세 번째이다.,neutral
27995,2997,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선은 흔히 비자림로라고 불린다.,entailment
27996,2998,흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.,비흡연자는 발코니 있는 방이 필요없습니다.,neutral


In [5]:
len(test)

1666

In [6]:
# step2
def chrome_setting():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    return driver

driver=chrome_setting()

In [7]:
data_path='./data/'

In [8]:
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

    target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

    for i in tqdm(range(start_index,final_index)): 

        if (i!=0)&(i%99==0):
            time.sleep(2)
            print('{}th : '.format(i), backtrans)
            np.save('./data/' +'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)

        try:
            driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
            time.sleep(1.5)
            element=WebDriverWait(driver, 10).until(target_present)
            time.sleep(0.1)
            backtrans = element.text 

            if (backtrans=='')|(backtrans==' '):
                element=WebDriverWait(driver, 10).until(target_present)
                backtrans = element.text 
                trans_list.append(backtrans)
            else:
                trans_list.append(backtrans)

        except:
            trans_list.append('')

In [9]:
# trans_list=[]
# kor_to_trans(train['premise'], 'en', 0, 10000)
# np.save(data_path+'kor_premise_to_eng_train_0_10000.npy',trans_list)

# trans_list=[]
# kor_to_trans(train['premise'], 'en', 10000, 20000)
# np.save(data_path+'kor_premise_to_eng_train_10000_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['premise'], 'en', 20000, len(train))
np.save(data_path+'kor_premise_to_eng_train_20000_all.npy',trans_list)

# trans_list=[]
# kor_to_trans(test['premise'], 'en', 0, len(test))
# np.save(data_path+'kor_premise_to_eng_test_0_all.npy',trans_list)

  1%|          | 97/7998 [02:48<3:47:30,  1.73s/it]

20097th :  The power of people abroad will be of great help to us.


  2%|▏         | 196/7998 [05:43<3:46:10,  1.74s/it]

20196th :  However, the Democratic People's Republic of Korea agreed to repatriate the body of its soldiers on the morning of the 14th, but has not responded.


  4%|▎         | 295/7998 [08:37<3:41:35,  1.73s/it]

...

 98%|█████████▊| 7819/7998 [3:47:57<05:09,  1.73s/it]

27819th :  However, it was too difficult to find at first.


 99%|█████████▉| 7918/7998 [3:50:49<02:17,  1.72s/it]

27918th :  Characters who are far from reality are annoying.


100%|██████████| 7998/7998 [3:53:09<00:00,  1.75s/it]


In [10]:
# trans_list=[]
# kor_to_trans(train['hypothesis'], 'en',0,10000)
# np.save(data_path+'kor_hypothesis_to_eng_train_0_10000.npy',trans_list)

# trans_list=[]
# kor_to_trans(train['hypothesis'], 'en',10000,20000)
# np.save(data_path+'kor_hypothesis_to_eng_train_10000_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['hypothesis'], 'en',20000,len(train))
np.save(data_path+'kor_hypothesis_to_eng_train_20000_all.npy',trans_list)

# trans_list=[]
# kor_to_trans(test['hypothesis'], 'en', 0, len(test))
# np.save(data_path+'kor_hypothesis_to_eng_test_0_all.npy',trans_list)

  1%|          | 97/7998 [02:56<3:46:45,  1.72s/it] 

20097th :  People abroad will not help us at all.


  2%|▏         | 196/7998 [05:49<3:43:42,  1.72s/it]

20196th :  There is a body of a soldier of the Democratic People's Republic of Korea who needs repatriation.


  4%|▎         | 295/7998 [08:42<3:44:53,  1.75s/it]

...

 98%|█████████▊| 7819/7998 [3:50:52<05:10,  1.74s/it]

27819th :  I had a hard time when I first visited.


 99%|█████████▉| 7918/7998 [3:54:07<02:18,  1.73s/it]

27918th :  I get annoyed when the character is far from reality.


100%|██████████| 7998/7998 [3:56:30<00:00,  1.77s/it]


In [11]:
eng_data0=np.load(data_path+'kor_premise_to_eng_train_0_10000.npy')
eng_data1=np.load(data_path+'kor_premise_to_eng_train_10000_20000.npy')
eng_data2=np.load(data_path+'kor_premise_to_eng_train_20000_all.npy')


eng_data=[*eng_data0,*eng_data1,*eng_data2]
eng_data_premise=pd.DataFrame(eng_data,columns=['eng_premise'])

In [12]:
eng_data0=np.load(data_path+'kor_hypothesis_to_eng_train_0_10000.npy')
eng_data1=np.load(data_path+'kor_hypothesis_to_eng_train_10000_20000.npy')
eng_data2=np.load(data_path+'kor_hypothesis_to_eng_train_20000_all.npy')


eng_data=[*eng_data0,*eng_data1,*eng_data2]
eng_dat_hypothesisa=pd.DataFrame(eng_data,columns=['eng_hypothesis'])

In [13]:
eng_train=pd.concat([train,eng_data_premise,eng_dat_hypothesisa],axis=1)
eng_train

,index,premise,hypothesis,label,eng_premise,eng_hypothesis
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,Ssireum is a representative game of men that h...,It's a play of women in Ssireum.
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,"Samsung said it was considering legal action, ...",There were three people who made their own plays.
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,"To this end, a predictive crime prevention sys...",This is because the purpose is to establish an...
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,Gwangju Metropolitan City has actively started...,The natives were satisfied with the comprehens...
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,If you really want to last a long time as a co...,"In this situation, very few companies show res..."
...,...,...,...,...,...,...
27993,2995,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사는 중단없이 마무리 되었다.,contradiction,Construction to expand Local Road 1112 commonl...,The construction of expanding Local Road 1112 ...
27994,2996,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선을 넓히는 공사가 중단된 건 세 번째이다.,neutral,Construction to expand Local Road 1112 commonl...,It is the third time that construction to expa...
27995,2997,흔히 비자림로라고 불리는 지방도 제1112호선을 넓히는 공사가 1년만에 재개되었다가...,지방도 제1112호선은 흔히 비자림로라고 불린다.,entailment,Construction to expand Local Road 1112 commonl...,Local Road 1112 is commonly referred to as Bij...
27996,2998,흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.,비흡연자는 발코니 있는 방이 필요없습니다.,neutral,Smokers can smoke on the balcony in a room wit...,Non-smokers do not need a room with a balcony.


In [14]:
eng_train = eng_train[['eng_premise','eng_hypothesis','label']]
eng_train

,eng_premise,eng_hypothesis,label
0,Ssireum is a representative game of men that h...,It's a play of women in Ssireum.,contradiction
1,"Samsung said it was considering legal action, ...",There were three people who made their own plays.,contradiction
2,"To this end, a predictive crime prevention sys...",This is because the purpose is to establish an...,entailment
3,Gwangju Metropolitan City has actively started...,The natives were satisfied with the comprehens...,neutral
4,If you really want to last a long time as a co...,"In this situation, very few companies show res...",neutral
...,...,...,...
27993,Construction to expand Local Road 1112 commonl...,The construction of expanding Local Road 1112 ...,contradiction
27994,Construction to expand Local Road 1112 commonl...,It is the third time that construction to expa...,neutral
27995,Construction to expand Local Road 1112 commonl...,Local Road 1112 is commonly referred to as Bij...,entailment
27996,Smokers can smoke on the balcony in a room wit...,Non-smokers do not need a room with a balcony.,neutral


In [15]:
eng_train.to_csv(f'./data/eng_train.csv', index=False)

In [16]:
eng_data0=np.load(data_path+'kor_premise_to_eng_test_0_all.npy')


eng_data=[*eng_data0]
eng_data_premise=pd.DataFrame(eng_data,columns=['eng_premise'])

In [17]:
eng_data0=np.load(data_path+'kor_hypothesis_to_eng_test_0_all.npy')


eng_data=[*eng_data0]
eng_data_hypothesis=pd.DataFrame(eng_data,columns=['eng_hypothesis'])

In [18]:
eng_test=pd.concat([test,eng_data_premise,eng_data_hypothesis],axis=1)
eng_test

,index,premise,hypothesis,label,eng_premise,eng_hypothesis
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,answer,"However, it is a little narrow, so it is incon...",There was enough space to unfold the care.
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,answer,And the location is closer to Xiaonanman stati...,You can arrive at Xiaonanmen Station before Si...
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,answer,Efforts to explain and understand every phrase...,"I talked a lot about something, but it didn't ..."
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,answer,"No matter how many times I watch it again, my ...","When I saw it again, I was calm."
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,answer,"If you apply for occupancy in August, young pe...","If you apply for occupancy in August, newlywed..."
...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,answer,"In addition, you can earn enough income throug...",Processing image data is the most difficult.
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,answer,"Looking at the ending, I think the second part...","Looking at the ending, there must be two films."
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,answer,"In a social distancing situation, it is said t...","In the situation of social distancing, high-ra..."
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,answer,I stayed here for 3 nights and 4 days in Rome.,I stayed here and visited Roman attractions.


In [19]:
eng_test = eng_test[['eng_premise','eng_hypothesis']]
eng_test

,eng_premise,eng_hypothesis
0,"However, it is a little narrow, so it is incon...",There was enough space to unfold the care.
1,And the location is closer to Xiaonanman stati...,You can arrive at Xiaonanmen Station before Si...
2,Efforts to explain and understand every phrase...,"I talked a lot about something, but it didn't ..."
3,"No matter how many times I watch it again, my ...","When I saw it again, I was calm."
4,"If you apply for occupancy in August, young pe...","If you apply for occupancy in August, newlywed..."
...,...,...
1661,"In addition, you can earn enough income throug...",Processing image data is the most difficult.
1662,"Looking at the ending, I think the second part...","Looking at the ending, there must be two films."
1663,"In a social distancing situation, it is said t...","In the situation of social distancing, high-ra..."
1664,I stayed here for 3 nights and 4 days in Rome.,I stayed here and visited Roman attractions.


In [20]:
eng_test.to_csv(f'./data/eng_test.csv', index=False)